In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import h5py
import os
import glob
import warnings
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

from morphogenesis.plot_utils import *
np.set_printoptions(precision=3, suppress=True)

warnings.filterwarnings('ignore')

/Users/jcolen/miniconda3/envs/fly_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Learning a model for myosin

In [5]:
from morphogenesis.library.composite_library import fill_group_info, collect_library, take_time_derivatives, decompose_library
from morphogenesis.library.extra_functions import *
        
libraries = [
    ('m_ij', '../flydrive/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP/derivative_library', 'tensor_library'),
    ('v', '../flydrive/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP/derivative_library', 'tensor_library'), # Flow couples to myosin
    ('c', '../flydrive/WT/Moesin-GFP/derivative_library', 'scalar_library'), # Auxiliary coupled control field
]
    
extra_functions = [
    lambda data: material_derivative_terms(data, key='m_ij'),
    lambda data: add_static_sources(data, couple='m_ij'),
    lambda data: symmetric_tensor_couple(data, ['m_ij', 'E']),
    lambda data: multiply_tensor_by_scalar(data, None, ['c']),
    lambda data: delete_non_tensors(data, like='m_ij'),
]

embryos = os.listdir(f'{libraries[0][1]}')
print(embryos)

with h5py.File('myosin_fitting.h5', 'w') as h5f:
    with tqdm(total=len(embryos)) as pbar:
        pbar.set_description('Collecting embryo data')
        for embryo in embryos:
            pbar.set_postfix(embryo=embryo)
            group, feature_names = fill_group_info(h5f, embryo, libraries)
            collect_library(group, libraries, feature_names, extra_functions=extra_functions)
            take_time_derivatives(group, 'm_ij', window_length=7)
            pbar.update()

['202007091200.h5', '202006261115.h5', '202008131005.h5', '202007081130.h5', '202007011145.h5', 'ensemble.h5', '202009041145.h5']


In [8]:
from morphogenesis.modeling.sindy_utils import *
from morphogenesis.modeling.fly_sindy import FlySINDy
import pysindy as ps

with h5py.File('myosin_fitting.h5', 'r') as h5f:
    X, X_dot, feature_names = collect_library_terms(h5f, 'm_ij', tmin=-5, tmax=10, feature_names=None)

sindy = FlySINDy(
    optimizer=ps.STLSQ(threshold=1e-2, alpha=1e1, normalize_columns=True),
    feature_names = feature_names,
    n_models=10,
    n_candidates_to_drop=0,
    subset_fraction=0.1)
sindy.fit(x=X, x_dot=X_dot)
print(f'\nThreshold = {1e-2:g}, Alpha = {1e1:g}')
sindy.print(lhs=['D_t m_ij'])


Threshold = 0.01, Alpha = 10
D_t m_ij = (0.095 + -0.075 c) Static_DV Tr(m_ij) + (-0.207 + 0.196 c) m_ij + (0.680 + 0.170 c) m_ij Tr(E) + (1.462 + -1.303 c) m_ij Tr(m_ij)


In [19]:
print(len(sindy.feature_names))
print(sindy.feature_names)

16
['E', 'E Tr(m_ij)', 'Static_DV', 'Static_DV Tr(m_ij)', 'c E', 'c E Tr(m_ij)', 'c Static_DV', 'c Static_DV Tr(m_ij)', 'c m_ij', 'c m_ij Tr(E)', 'c m_ij Tr(m_ij)', 'c {m_ij, E}', 'm_ij', 'm_ij Tr(E)', 'm_ij Tr(m_ij)', '{m_ij, E}']


In [9]:
'''
Repeat the fitting process restricted to the library terms above
We do this because ensembling suppresses coefficients due to subleading terms!
'''
with h5py.File('myosin_fitting.h5', 'r') as h5f:
    X, X_dot, feature_names = collect_library_terms(h5f, 'm_ij', tmin=-5, tmax=10, feature_names=overleaf_feature_names)


#No longer threshold since we've established the above terms
sindy = FlySINDy(
    optimizer=ps.STLSQ(threshold=0, alpha=0, normalize_columns=True),
    feature_names = feature_names,
    n_models=1,
    subset_fraction=0.1)
sindy.fit(x=X, x_dot=X_dot)
print(f'\nThreshold = {0:g}, Alpha = {0:g}')
sindy.print(lhs=['D_t m_ij'])


Threshold = 0, Alpha = 0
D_t m_ij = (-0.217 + 0.202 c) m_ij + (1.547 + -1.365 c) m_ij Tr(m_ij) + (0.664 + 0.202 c) m_ij Tr(E) + (0.095 + -0.073 c) Static_DV Tr(m_ij)


# Learning a model for actin

In [10]:
from morphogenesis.library.composite_library import fill_group_info, collect_library, take_time_derivatives, decompose_library
from morphogenesis.library.extra_functions import *
libraries = [
    ('c', '../flydrive/WT/Moesin-GFP/derivative_library', 'scalar_library'), #Want cadherin dynamics
    ('m_ij', '../flydrive/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP/derivative_library', 'tensor_library'), #Not focused on myosin dynamics
    ('v', '../flydrive/WT/Moesin-GFP/derivative_library', 'tensor_library'), #Not focused on flow dynamics
]

    
extra_functions = [
    lambda data: material_derivative_terms(data, key='c'),
    lambda data: add_v_squared(data),
    lambda data: tensor_trace(data, keys=['m_ij', 'E']),
    lambda data: scalar_couple(data, keys=['c', 'Tr(E)', 'Tr(m_ij)']),
    lambda data: delete_high_order_scalars(data, max_space_order=1),
    lambda data: delete_non_scalars(data, like='c'),
]

embryos = os.listdir(f'{libraries[0][1]}')
print(embryos)

with h5py.File('actin_fitting.h5', 'w') as h5f:
    with tqdm(total=len(embryos)) as pbar:
        pbar.set_description('Collecting embryo data')
        for embryo in embryos:
            pbar.set_postfix(embryo=embryo)
            group, feature_names = fill_group_info(h5f, embryo, libraries)
            collect_library(group, libraries, feature_names, extra_functions=extra_functions)
            take_time_derivatives(group, 'c', window_length=7)
            pbar.update()

['202001231400.h5', '202001262040.h5', '202001271215.h5', '202001231555.h5', '202001101740.h5', '202001271700.h5', '202002111638.h5', 'ensemble.h5', '202001121330.h5', '202001261520.h5']


In [11]:
from morphogenesis.modeling.sindy_utils import *
from morphogenesis.modeling.fly_sindy import FlySINDy
import pysindy as ps

with h5py.File('actin_fitting.h5', 'r') as h5f:
    X, X_dot, feature_names = collect_library_terms(h5f, 'c', tmin=-5, tmax=10, feature_names=None)

sindy = FlySINDy(
    optimizer=ps.STLSQ(threshold=1e-2, alpha=1e1, normalize_columns=True),
    feature_names = feature_names,
    n_models=10,
    n_candidates_to_drop=0,
    subset_fraction=0.1)
sindy.fit(x=X, x_dot=X_dot)
print(f'\nThreshold = {1e-2:g}, Alpha = {1e1:g}')
sindy.print(lhs=['D_t c'])

  0%|          | 0/10 [00:00<?, ?it/s]


Threshold = 0.01, Alpha = 10
D_t c = 0
